In [1]:
#Imports
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
#Read the CSV & Create DF
filtered_df = pd.read_csv("routine_boulder_inspections_filtered.csv")
filtered_df.head()

,Unnamed: 0,facilityId,facilityName,siteAddress,streetNumber,streetDirection,streetName,streetType,streetUnit,city,...,categoryOfFacility,inspectionType,inspectionDate,violationCode,violation,violationPoints,violationType,violationStatus,inspectionScore,location
0,0,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,01B,Wholesome Free of Spoilage,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
1,1,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,15B,Linen Properly Stored,0,Non-Critical,Not Observed,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
2,2,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,06C,Soap and Drying Devices,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
3,3,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,2/6/2015,03G,Adequate Equipment to Maintain Food Temperatures,0,Critical,In,0,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
4,4,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,03C,Hot Hold at 135 Degrees Farenheit or Greater,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."


In [8]:
#determine which field or fields should be used in geocoding.
filtered_df.isnull().sum()
# siteAddress,city,state

Unnamed: 0                 0
facilityId                 0
facilityName               0
siteAddress                0
streetNumber             124
streetDirection       170538
streetName                 0
streetType              5003
streetUnit            150059
city                       0
state                      0
zip                        0
typeOfFacility             0
categoryOfFacility         0
inspectionType             0
inspectionDate             0
violationCode              0
violation                  0
violationPoints            0
violationType          16713
violationStatus         1476
inspectionScore            0
location                   0
dtype: int64

In [16]:
#drop 'unnamed' column
unnamed_dropped = filtered_df.drop('Unnamed: 0',1)

# concatenate address, add to a new column

# new dataframe containing all UNIQUE addresses
# unnamed_dropped.head()

C:\Users\natal\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,facilityId,facilityName,siteAddress,streetNumber,streetDirection,streetName,streetType,streetUnit,city,state,...,categoryOfFacility,inspectionType,inspectionDate,violationCode,violation,violationPoints,violationType,violationStatus,inspectionScore,location
0,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,CO,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,01B,Wholesome Free of Spoilage,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
1,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,CO,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,15B,Linen Properly Stored,0,Non-Critical,Not Observed,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
2,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,CO,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,06C,Soap and Drying Devices,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
3,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,CO,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,2/6/2015,03G,Adequate Equipment to Maintain Food Temperatures,0,Critical,In,0,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."
4,FA0003323,RUNZA,1743 MAIN ST,1743,NaN,MAIN,ST,NaN,LONGMONT,CO,...,FULL MENU LIMITED SERVICE,ROUTINE INSPECTION,11/13/2013,03C,Hot Hold at 135 Degrees Farenheit or Greater,0,Critical,In,27,"1743 MAIN ST\r\nLONGMONT, CO 80501\r\n(40.1897..."


In [22]:
# struture #API Call & test on a single address
params = {
    "address": "1743 MAIN ST LONGMONT CO USA",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

latlong = requests.get(base_url, params=params).json()

latlong ["results"][0]["location"]

{'results': [{'address_components': [{'long_name': '1743',
     'short_name': '1743',
     'types': ['street_number']},
    {'long_name': 'Main Street', 'short_name': 'Main St', 'types': ['route']},
    {'long_name': 'Longmont',
     'short_name': 'Longmont',
     'types': ['locality', 'political']},
    {'long_name': 'Boulder County',
     'short_name': 'Boulder County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'Colorado',
     'short_name': 'CO',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']},
    {'long_name': '80501', 'short_name': '80501', 'types': ['postal_code']},
    {'long_name': '2035',
     'short_name': '2035',
     'types': ['postal_code_suffix']}],
   'formatted_address': '1743 Main St, Longmont, CO 80501, USA',
   'geometry': {'bounds': {'northeast': {'lat': 40.18985139999999,
      'lng': -105.1026154},
     'southwest':

In [ ]:
#create forloop(s) to extract each piece of data and drop into the url line by line?

#iterate through that dataframe!
for index, row in _________.iterrows():
    # Get the Address from the dataframe
    address = row[__________]
    params["address"] = address
#     # Add the latitude and longitude to location key for the params dictionary.
#     params["location"] = f"{lat},{lng}"
    
    # Set the base URL to grab JSON data
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    # Make request and retrieve the JSON data from the search. 
    latlong = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Get coordinates & store, if not found, skip! 
    try: 
        ___________________df.loc[index, "Coords"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping") 

In [ ]:
#Make the f&*^*ng call 

In [ ]:
#create a new data frame with original address and lat/longs

In [ ]:
#export combined CSV

In [ ]:
#merge with the original dataset?

In [ ]:
#export combined CSV